In [2]:
from binance.spot import Spot
from datetime import datetime,timedelta
client = Spot()

In [4]:
from sagemaker.feature_store.feature_group import FeatureGroup
import boto3
import pandas as pd
import numpy as np
import io
from sagemaker.session import Session
from sagemaker import get_execution_role

# customers_feature_group_name = 'BtcPriceData' 
# customers_feature_group = FeatureGroup(
#     name=customers_feature_group_name, sagemaker_session=sagemaker_session
#)

feature_group = FeatureGroup(name = 'BtcPriceData')


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/berkan/Library/Application Support/sagemaker/config.yaml


In [7]:
def get_ts_for_days():
    start_date = datetime(2024, 1, 10)
    end_date = datetime(2024, 1, 10)
    day_count = (end_date - start_date).days + 1
    for  n in range(day_count):
        temp_date = start_date + timedelta(days = n)
        print('date',temp_date)
        yield temp_date, int(datetime.timestamp(temp_date)*1000)


In [8]:
for date, timestamp in get_ts_for_days():
    klines = client.klines("BTCUSDT", "5m",limit = 11*12  ,startTime = timestamp) #endTime = end_ts )#,limit = 1)#24 * 4)
    df = pd.DataFrame(klines, columns = ['open_time','open_price','high_price','low_price','close_price','volume','close_time','quote_asset_volume','number_of_trades','taker_buy_base_asset_volume','taker_buy_quote_asset_volume','ignore'])
    #df['open_time'] = pd.to_datetime(df['open_time'], unit='ms').dt.strftime('%Y-%m-%d %T')
    #df['close_time'] = pd.to_datetime(df['close_time'], unit='ms').dt.strftime('%Y-%m-%d %T')
    #df['EventTime'] = round(time.time())
    df['close_time'] =  df['close_time'].div(1000)
    df = df.drop(columns = ['ignore','taker_buy_quote_asset_volume'])
    df = df.astype(float)
    #print(df)
    #feature_group.load_feature_definitions(df)
    feature_group.ingest(data_frame=df, max_workers=2, wait=True)


date 2024-01-10 00:00:00


In [ ]:
def get_ts_for_days():
    start_date = datetime(2024, 1, 3)
    end_date = datetime(2024, 1, 7)
    day_count = (end_date - start_date).days + 1
    for  n in range(day_count):
        temp_date = start_date + timedelta(days = n)
        print('date',temp_date)
        yield temp_date, int(datetime.timestamp(temp_date)*1000)


In [5]:
klines = client.klines("BTCUSDT", "5m",limit = 2*12 ) #endTime = end_ts )#,limit = 1)#24 * 4)
df = pd.DataFrame(klines, columns = ['open_time','open_price','high_price','low_price','close_price','volume','close_time','quote_asset_volume','number_of_trades','taker_buy_base_asset_volume','taker_buy_quote_asset_volume','ignore'])
#df['open_time'] = pd.to_datetime(df['open_time'], unit='ms').dt.strftime('%Y-%m-%d %T')
#df['close_time'] = pd.to_datetime(df['close_time'], unit='ms').dt.strftime('%Y-%m-%d %T')
#df['EventTime'] = round(time.time())
df['close_time'] =  df['close_time'].div(1000)
df = df.drop(columns = ['ignore','taker_buy_quote_asset_volume'])
df = df.astype(float)
#print(df)
#feature_group.load_feature_definitions(df)
#feature_group.ingest(data_frame=df, max_workers=2, wait=True)
df

,open_time,open_price,high_price,low_price,close_price,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume
0,1.705001e+12,46374.29,46396.84,46283.14,46339.49,230.15640,1.705001e+09,1.066271e+07,7897.0,101.50257
1,1.705001e+12,46339.49,46552.18,46338.80,46534.02,113.29337,1.705001e+09,5.263651e+06,6098.0,59.61642
2,1.705001e+12,46534.02,46553.01,46483.24,46553.00,147.64330,1.705002e+09,6.867545e+06,5884.0,61.74765
3,1.705002e+12,46553.01,46632.02,46536.71,46600.00,113.20626,1.705002e+09,5.274323e+06,6489.0,40.95067
4,1.705002e+12,46600.00,46628.59,46591.30,46612.98,114.16860,1.705002e+09,5.321323e+06,4822.0,45.97281
5,1.705002e+12,46612.98,46711.09,46609.97,46701.21,203.51323,1.705003e+09,9.499861e+06,7923.0,110.67580
6,1.705003e+12,46701.20,46701.21,46580.00,46581.99,100.84426,1.705003e+09,4.701469e+06,5640.0,32.77103
7,1.705003e+12,46581.99,46735.29,46581.99,46692.30,147.55739,1.705003e+09,6.889796e+06,5801.0,58.23734
8,1.705003e+12,46692.29,46888.00,46692.29,46813.82,444.09780,1.705003e+09,2.078509e+07,14541.0,225.47821
9,1.705004e+12,46813.82,46921.61,46756.00,46921.61,361.27986,1.705004e+09,1.692685e+07,9492.0,183.04449


In [6]:
feature_group.ingest(data_frame=df, max_workers=2, wait=True)

IngestionManagerPandas(feature_group_name='BtcPriceData', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x11eb4d990>, sagemaker_session=<sagemaker.session.Session object at 0x1085f0a90>, max_workers=2, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x11ef9f880>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])